In [4]:
from keras.models import load_model
import cv2
import numpy as np



# تحميل النموذج المدرب مسبقًا بدون إعادة تجميع
model = load_model('emotion_model.hdf5', compile=False)

# تسميات المشاعر
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# تحميل كاشف الوجوه من OpenCV
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# فتح الكاميرا (0 تعني الكاميرا الافتراضية)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ لم يتم فتح الكاميرا.")
    exit()

while True:
    # التقاط الإطار الحالي من الكاميرا
    ret, frame = cap.read()
    if not ret:
        break

    # تحويل إلى تدرجات الرمادي
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # اكتشاف الوجوه
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        # قص الوجه وتحضيره للنموذج
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (64, 64))
        roi = np.expand_dims(roi_gray, axis=(0, -1)) / 255.0

        # التنبؤ بالمشاعر
        prediction = model.predict(roi, verbose=0)
        emotion = emotion_labels[np.argmax(prediction)]

        # رسم مستطيل واسم الشعور
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(frame, emotion, (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # عرض الإطار
    cv2.imshow("Real-time Emotion Detection", frame)

    # الخروج عند الضغط على 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# إغلاق الكاميرا والنوافذ
cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'keras'